# Titanic Dataset Preprocessing (for Decision Tree Model)

## 1. Giới Thiệu
**Mục tiêu:** Làm sạch và xử lý dữ liệu Titanic để phục vụ mô hình *Decision Tree* dự đoán khả năng sống sót.

**Bước thực hiện:**
1. Đọc và khám phá dữ liệu gốc (`train.csv`)
2. Xử lý giá trị thiếu (đặc biệt là `Age`, `Cabin`, `Embarked`)
3. Mã hóa biến phân loại (`Sex`, `Embarked`)
4. Chọn đặc trưng phù hợp cho mô hình
5. Lưu dữ liệu sau tiền xử lý

## 2. Import thư viện và đọc dữ liệu

In [7]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Đọc dữ liệu Titanic
data = pd.read_csv("train.csv")

# Hiển thị thông tin cơ bản
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


**Giải thích:**
- Dữ liệu có 891 hàng và 12 cột.
- Các cột quan trọng: `Survived`, `Pclass`, `Sex`, `Age`, `SibSp`, `Parch`, `Fare`, `Cabin`, `Embarked`.
- Cột `Cabin` có nhiều giá trị bị thiếu → cần xem xét loại bỏ.

## 3. Phân tích sơ bộ dữ liệu & kiểm tra giá trị thiếu

In [8]:
# Thống kê mô tả
display(data.describe())

# Kiểm tra giá trị thiếu
data.isnull().sum()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

**Giải thích:**
- `Cabin` có hơn 77% giá trị bị thiếu → loại bỏ.
- `Age` có khoảng 20% giá trị bị thiếu → điền theo trung vị nhóm Pclass + Sex.
- `Embarked` thiếu 2 giá trị → điền bằng giá trị xuất hiện nhiều nhất (mode).

## 4. Xử lý giá trị thiếu

In [9]:
# Bỏ cột Cabin do quá nhiều giá trị thiếu
data.drop(columns=['Cabin'], inplace=True)

# Điền Age theo trung vị của từng nhóm (Pclass, Sex)
data['Age'] = data.groupby(['Pclass', 'Sex'])['Age'].transform(lambda x: x.fillna(x.median()))

# Điền Embarked bằng giá trị xuất hiện nhiều nhất
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

# Kiểm tra lại giá trị thiếu
data.isnull().sum()


C:\Users\HOME\AppData\Local\Temp\ipykernel_14016\458150985.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

✅ **Kết quả cuối:**
Tất cả giá trị thiếu (`NaN`) đã được xử lý.  
Cột `Cabin` bị loại bỏ hoàn toàn khỏi tập dữ liệu.

## 5. Mã hóa biến phân loại

In [ ]:
# Mã hóa giới tính (male = 0, female = 1)
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

# Mã hóa Embarked thành biến giả (dummy variables)
data = pd.get_dummies(data, columns=['Embarked'], drop_first=True)

# Kiểm tra dữ liệu sau mã hóa
data.head()

**Giải thích:**
- `Sex` được mã hóa thành 0/1 để mô hình hiểu.
- `Embarked` được chuyển thành 2 cột mới: `Embarked_Q`, `Embarked_S`.
- Dữ liệu giờ chỉ còn dạng số → sẵn sàng cho huấn luyện.

## 6. Chuẩn bị dữ liệu cho mô hình

In [ ]:
# Chọn đặc trưng (features) và nhãn (target)
X = data.drop(columns=['Survived', 'Name', 'Ticket', 'PassengerId'])
y = data['Survived']

print("✅ Dữ liệu đầu vào (X):", X.shape)
print("✅ Nhãn mục tiêu (y):", y.shape)

**Giải thích:**
- Các cột như `Name`, `Ticket`, `PassengerId` không ảnh hưởng đến việc dự đoán → loại bỏ.
- Dữ liệu còn lại chứa các đặc trưng hữu ích cho mô hình cây quyết định.

## 7. Lưu data mới sau khi xử lý

In [11]:
# Tạo thư mục nếu chưa có
import os
os.makedirs("data", exist_ok=True)

# Lưu dữ liệu sau tiền xử lý
data.to_csv("data/processed_titanic.csv", index=False)
print("📁 Dữ liệu đã được lưu tại: data/processed_titanic.csv")

# Kiểm tra dữ liệu sau khi lưu
preview = pd.read_csv("data/processed_titanic.csv")
print("📄 5 dòng đầu trong file đã lưu:")
display(preview.head())

print("\n📊 Thông tin dữ liệu sau xử lý:")
preview.info()


📁 Dữ liệu đã được lưu tại: data/processed_titanic.csv
📄 5 dòng đầu trong file đã lưu:


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S



📊 Thông tin dữ liệu sau xử lý:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


✅ **Giải thích:**
- Dữ liệu sạch được lưu để sử dụng ở bước tiếp theo (huấn luyện mô hình).
- File `data/processed_titanic.csv` sẽ được nạp vào trong notebook *Decision Tree Training*.

# Tổng Kết
### 🧩 Tóm tắt Tiền Xử Lý

| Bước | Mục tiêu | Kết quả |
|------|-----------|----------|
| 1 | Loại bỏ `Cabin` | Tránh nhiễu do dữ liệu thiếu |
| 2 | Điền `Age` theo nhóm | Giảm bias, giữ thông tin quan trọng |
| 3 | Mã hóa `Sex`, `Embarked` | Chuẩn hóa dữ liệu đầu vào |
| 4 | Lưu dữ liệu | Tập dữ liệu sẵn sàng huấn luyện mô hình Decision Tree |
